In [1]:
import pandas as pd
import pulp
import re

In [2]:
salary_cap = 194
roster_spots = {'QB':2
                ,'WR':4
                ,'TE':2
                ,'RB':2
                ,'KR':1
                ,'DE':1
                }

In [3]:
league_df = pd.read_csv('data.csv')

#Get position
league_df['position'] = league_df['Converted'].str[:2]

league_df.head()

,Converted,Points,Exp_Cost,23_Cost,Max_Cost,Player,position
0,QB01,461.60,50,51,51,Jalen Hurts,QB
1,QB02,436.63,47,50,50,Josh Allen,QB
2,QB03,413.86,44,49,49,Lamar Jackson,QB
3,QB04,391.19,41,48,48,Patrick Mahomes,QB
4,QB05,366.90,34,37,37,Anthony Richardson,QB


In [4]:
salaries = {}
points = {}
for pos in league_df['position'].unique():
    available_pos = league_df[league_df['position']==pos]
    salary = list(available_pos[['Converted','Exp_Cost']].set_index('Converted').to_dict().values())[0]
    point = list(available_pos[['Converted','Points']].set_index('Converted').to_dict().values())[0]
    salaries[pos] = salary
    points[pos] = point

In [ ]:
for k,v in points.items():
    x = pulp.LpVariable.dict(k,v)
    print(k,v)

In [ ]:
_vars = {k: pulp.LpVariable.dict(k, v, cat="Binary") for k, v in points.items()}

In [ ]:
_vars

In [ ]:
prob = pulp.LpProblem("Fantasy", pulp.LpMaximize)
rewards = []
costs = []
position_constraints = []
# Setting up the reward
for k, v in _vars.items():
    costs += pulp.lpSum([salaries[k][i] * _vars[k][i] for i in v])
    rewards += pulp.lpSum([points[k][i] * _vars[k][i] for i in v])
    prob += pulp.lpSum([_vars[k][i] for i in v]) <= roster_spots[k]
    
prob += pulp.lpSum(rewards)
prob += pulp.lpSum(costs) <= salary_cap

In [ ]:
prob.solve()

In [ ]:
def summary(prob):
    div = '---------------------------------------\n'
    print("Variables:\n")
    score = str(prob.objective)
    constraints = [str(const) for const in prob.constraints.values()]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
        if v.varValue != 0:
            print(v.name, "=", v.varValue)
    print(div)
    print("Constraints:")
    for constraint in constraints:
        constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
        if constraint_pretty != "":
            print("{} = {}".format(constraint_pretty, eval(constraint_pretty)))
    print(div)
    print("Score:")
    score_pretty = " + ".join(re.findall("[0-9\.]+\*1.0", score))
    print("{} = {}".format(score_pretty, eval(score)))

In [ ]:
summary(prob)

In [5]:
league_df

,Converted,Points,Exp_Cost,23_Cost,Max_Cost,Player,position
0,QB01,461.60,50,51,51,Jalen Hurts,QB
1,QB02,436.63,47,50,50,Josh Allen,QB
2,QB03,413.86,44,49,49,Lamar Jackson,QB
3,QB04,391.19,41,48,48,Patrick Mahomes,QB
4,QB05,366.90,34,37,37,Anthony Richardson,QB
...,...,...,...,...,...,...,...
253,DE08,133.67,2,1,1,Kansas City Chiefs,DE
254,DE09,130.33,2,1,1,Indianapolis Colts,DE
255,DE10,128.67,1,1,1,San Francisco 49ers,DE
256,DE11,126.33,1,1,1,Buffalo Bills,DE


In [17]:
league_df[league_df['Player']== 'Josh Allen']['position'].tolist()[0]

'QB'

1    QB
Name: position, dtype: object
